# Top 10 Neighborhoods with Vegetarian-Friendly Dining in Toronto, Canada

## *Background*

Hi, my name is Anthony and I have been a vegetarian for over 4 years. While society has made a lot of advancements in being more inclusive for those with vegetarian diets, there are still some places that are better than others. After I graduated college, one of my best friends moved to Toronto and she is also a vegetarian. Before her move we spoke about vegetarian options in Toronto and would have benefited from knowing the top neighborhoods with vegetarian-friendly dining.

## *Problem Statement*

Find the top 10 neighborhoods in Toronto, Canada with vegetarian-friendly dining options.

People with vegetarian diets can often find themselves in a bad situation if they do not plan out their moves because there is great variance between cities in terms of vegetarian-friendly dining options. 

## *Stakeholders*

The number of those electing to adopt a vegetarian diet is growing annually. The people who are stakeholders in this problem are:
 + Those with a vegetarian diet currently living in Toronto.
 + Those with a vegetarian diet planning on moving to Toronto.
 + Businesses and restaurants who want to target a vegetarian-based audience in Toronto.
 + The City of Toronto to provide resources to those with a vegetarian diet.
 
## *How can we use data to solve the problem?*
 
We need two main sources of data in order to perform our analysis:
   1. List of neighborhoods in Toronto,
   2. List of vegetarian-friendly restaurants in Toronto.
   
The neighborhood data can be retrieved from Wikipedia using a webscraper (BeautifulSoup) to transform the data into a dataframe to be manipulated in pandas.

The list of vegetarian-friendly restaurants can be retrieved using Foursquare's API, this will ensure we have the most up-to-date information as restaurants are constantly being built. Having up-to-date information in regards to the growing (or shriving) number of restaurants overtime will be valuable for businesses who are targeting vegetarian audiences.

With both of these datasets, we will be able to perform k-means clustering. The purpose of k-means clustering is to cluster all the vegetarian friendly restaurants in Toronto and use the optimal cluster as a baseline coordinate to find the best cities.

### Wikipedia Data listing Toronto Neighborhoods

In the code below, you can see the data that will be used to map out the different neighborhoods in Toronto. Remember, our goal is to find the top 10 friendliest neighborhoods for vegetarian dining. 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(url)
soup = BeautifulSoup(wiki.text, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Foursquare data listing vegetarian friendly dining establishments

As shown below, the data will be called using Foursquare's API to ensure we constantly are feeding our analysis with the most up-to-date information. We will use this data to see where the best cities for vegetarian dining are by clustering.

In [5]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='ZOYDWVFMZWWEBXUD0HVA0QIO3UAJKJOQLCTZNWV3APPTKKEV',
client_secret='F5WNV4XRS3MA02DBDCN5ZXGYSOZHHP4GF5PQNHW1D4ODNLBC',
v='20180323',
ll='43.6532,-79.3832', #lat lng of Toronto
query='vegetarian',
limit=6000,
radius = 7000
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

df1 = pd.json_normalize(data['response'], record_path=['groups', 'items'], meta=[])
df1.rename(columns={'venue.location.lat':'lat',
                  'venue.location.lng':'lng'}, inplace=True)
df1[['venue.name', 'lat', 'lng']].head(5)

,venue.name,lat,lng
0,Fresh On Front,43.647815,-79.374453
1,Karine's,43.653699,-79.390743
2,Vegetarian Haven,43.656016,-79.392758
3,Buddha's Vegetarian,43.651904,-79.403312
4,Fresh On Spadina,43.648048,-79.396008
